In [12]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace,WebSearchTool, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
import os
from pydantic import BaseModel
from agents.model_settings import ModelSettings
from datetime import datetime

In [13]:
from search_file import search_file
from write_to_file import write_to_file
from clipboard_manager import save_clipboard, get_clipboard_history, get_last_clipboard
from wikipedia_tool import search_wikipedia
from docker_runner import run_code_in_docker
from send_email import send_email
from api_tester import api_request
from system_control import system_control
from extract_text import extract_text
from songplay import play_youtube
from scraper import scrape_dynamic_website
from runcommand import run_command
from calendar_event import manage_event
from launch_app import launch_app
from transcribe import record_audio,transcribe_audio
from text_to_speech import text_to_speech

In [14]:
from instructions import send_email_instruction, calendar_instruction,email_instruction, powershell_command_runner_instruction, web_scraper_instruction, system_control_instruction, api_tester_instruction, doc_reader_instruction, code_helper_instruction, websearch_instruction, app_launcher_instruction, file_searcher_writer_instruction, clipboard_manager_instruction

In [15]:
load_dotenv(override=True)

True

In [16]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI


In [17]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

### SYSTEM TOOLS

In [18]:
APP_LAUNCHER_AGENT=Agent(name="Gemini App laucher", instructions=app_launcher_instruction, model=gemini_model, tools=[launch_app])

In [19]:
FILE_SEARCHER_AGENT=Agent(name="Gemini Folder/File searcher and Writer", instructions=file_searcher_writer_instruction, model="gpt-4o-mini", tools=[search_file,write_to_file])

In [20]:
CLIPBOARD_MANAGER=Agent(name="Gemini  clipboard manager", instructions=clipboard_manager_instruction, model=gemini_model, tools=[save_clipboard, get_clipboard_history, get_last_clipboard],)

In [21]:
EMAIL_SENDER=Agent(name="Gemini  email sender", instructions=send_email_instruction, model=gemini_model, tools=[send_email],)

### INTERNET TOOLS

In [22]:
SEARCH_AGENT = Agent(
    name="Search agent",
    instructions=websearch_instruction,
    tools=[WebSearchTool(search_context_size="low"),search_wikipedia],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [23]:
CODING_AGENT = Agent(
    name="coding agent",
    instructions=code_helper_instruction,
    tools=[run_code_in_docker],
    model=gemini_model,

)

In [24]:
EMAIL_AGENT = Agent(
    name="email agent",
    instructions=email_instruction,
    model=gemini_model,
)

In [25]:
# message = "write an email saying hurraayyy!!!! and send that email to yourzlove56@gmail.com"

# with trace("EMAIL_AGENT"):
#     result = await Runner.run(EMAIL_AGENT, message)

In [26]:
DOCUMENT_AGENT = Agent(
    name="document agent",
    instructions=doc_reader_instruction,
    tools=[extract_text],
    model=gemini_model,
)

In [27]:
API_TESTER_AGENT = Agent(
    name="api agent",
    instructions=api_tester_instruction,
    tools=[api_request],
    model=gemini_model,
)

In [28]:
SYSTEM_CONTROL_AGENT = Agent(
    name="system control agent",
    instructions=system_control_instruction,
    tools=[system_control,play_youtube],
    model='gpt-4o-mini',
)

In [29]:
WEB_SCRAPER_AGENT = Agent(
    name="web scraper agent",
    instructions=web_scraper_instruction,
    tools=[scrape_dynamic_website],
    model=gemini_model,
)

In [30]:
POWERSHELL_AGENT = Agent(
    name="powershell agent",
    instructions=powershell_command_runner_instruction,
    tools=[run_command],
    model=gemini_model,
)

In [31]:

CALENDAR_AGENT = Agent(
    name="calendar agent",
    instructions=calendar_instruction + "\nCurrent date and time: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    tools=[manage_event],
    model=gemini_model,
)

In [32]:
agent_tools = [
    APP_LAUNCHER_AGENT.as_tool(
        tool_name="launch_app", 
        tool_description="Launch applications and programs on the system"
    ),
    
    FILE_SEARCHER_AGENT.as_tool(
        tool_name="file_operations",
        tool_description="Search for files and folders, read file contents, and write to files"
    ),
    
    CLIPBOARD_MANAGER.as_tool(
        tool_name="clipboard_manager",
        tool_description="Manage clipboard operations including saving, retrieving history, and getting recent clipboard content"
    ),
    
    SEARCH_AGENT.as_tool(
        tool_name="web_search",
        tool_description="Search the web and Wikipedia for information and answers to questions"
    ),
    
    CODING_AGENT.as_tool(
        tool_name="code_executor",
        tool_description="Execute and run code in a secure Docker environment for programming tasks"
    ),
    
    EMAIL_AGENT.as_tool(
        tool_name="email_sender",
        tool_description="Send emails by taking the email address and handle email-related tasks"
    ),
    
    DOCUMENT_AGENT.as_tool(
        tool_name="document_reader",
        tool_description="Extract and read text from documents, PDFs, and various file formats"
    ),
    
    API_TESTER_AGENT.as_tool(
        tool_name="api_tester",
        tool_description="Test APIs, make HTTP requests, and handle API interactions"
    ),
    
    SYSTEM_CONTROL_AGENT.as_tool(
        tool_name="system_control",
        tool_description="Control system functions and play YouTube videos"
    ),
    
    WEB_SCRAPER_AGENT.as_tool(
        tool_name="web_scraper",
        tool_description="Scrape and extract data from dynamic websites and web pages"
    ),
    
    POWERSHELL_AGENT.as_tool(
        tool_name="powershell_runner",
        tool_description="Execute PowerShell commands and run system-level operations"
    ),
    
    CALENDAR_AGENT.as_tool(
        tool_name="calendar_manager",
        tool_description="Create, update or delete events on calendar and give response"
    ),
    EMAIL_SENDER.as_tool(
        tool_name="email_sender",
        tool_description="Send emails by taking the email address and subject + body"
    )
]


In [34]:
message = "search about latest news of AI on wikipedia ,write an email and send that email to yourzlove56@gmail.com"

with trace("Final_agent"):
    result = await Runner.run(Final_agent, message)

In [37]:
print(result.final_output)

I have sent the email to yourzlove56@gmail.com regarding the latest news on Artificial Intelligence. If you need anything else, feel free to ask!


In [46]:
import asyncio
import threading
from IPython.display import display, HTML, clear_output
from httpx import stream
import ipywidgets as widgets
from io import StringIO
import sys
import time
import random

class StreamCapture:
    def __init__(self):
        self.output = StringIO()
        self.original_stdout = sys.stdout
    
    def start_capture(self):
        sys.stdout = self.output
    
    def stop_capture(self):
        sys.stdout = self.original_stdout
        content = self.output.getvalue()
        self.output.truncate(0)
        self.output.seek(0)
        return content

class ChatInterface:
    def __init__(self):
        # Convert all agents to tools
        agent_tools = [
            APP_LAUNCHER_AGENT.as_tool(
                tool_name="launch_app",
                tool_description="Launch applications and programs on the system"
            ),
            FILE_SEARCHER_AGENT.as_tool(
                tool_name="file_operations",
                tool_description="Search for files and folders, read file contents, and write to files"
            ),
            CLIPBOARD_MANAGER.as_tool(
                tool_name="clipboard_manager",
                tool_description="Manage clipboard operations including saving, retrieving history, and getting recent clipboard content"
            ),
            SEARCH_AGENT.as_tool(
                tool_name="web_search",
                tool_description="Search the web and Wikipedia for information and answers to questions"
            ),
            CODING_AGENT.as_tool(
                tool_name="code_executor",
                tool_description="Execute and run code in a secure Docker environment for programming tasks"
            ),
            EMAIL_AGENT.as_tool(
                tool_name="email_sender",
                tool_description="Send emails and handle email-related tasks"
            ),
            DOCUMENT_AGENT.as_tool(
                tool_name="document_reader",
                tool_description="Extract and read text from documents, PDFs, and various file formats"
            ),
            API_TESTER_AGENT.as_tool(
                tool_name="api_tester",
                tool_description="Test APIs, make HTTP requests, and handle API interactions"
            ),
            SYSTEM_CONTROL_AGENT.as_tool(
                tool_name="system_control",
                tool_description="Control system functions and play YouTube videos"
            ),
            WEB_SCRAPER_AGENT.as_tool(
                tool_name="web_scraper",
                tool_description="Scrape and extract data from dynamic websites and web pages"
            ),
            POWERSHELL_AGENT.as_tool(
                tool_name="powershell_runner",
                tool_description="Execute PowerShell commands and run system-level operations"
            ),
            CALENDAR_AGENT.as_tool(
                tool_name="calendar_manager",
                tool_description="Create, update or delete events on calendar and give response"
            )
        ]
        
        self.agent = Agent(
            name="Aether",
            tools=agent_tools,
            instructions="""You are Aether, an advanced AI assistant with access to a comprehensive suite of specialized tools. You are sophisticated, helpful, and speak with intelligence and elegance befitting your ethereal nature.

**Available Tools:**
- **launch_app**: Launch applications and programs on the system
- **file_operations**: Search for files and folders, read file contents, and write to files
- **clipboard_manager**: Manage clipboard operations including saving, retrieving history, and getting recent clipboard content
- **web_search**: Search the web and Wikipedia for information and answers to questions
- **code_executor**: Execute and run code in a secure Docker environment for programming tasks
- **email_sender**: Send emails and handle email-related tasks
- **document_reader**: Extract and read text from documents, PDFs, and various file formats
- **api_tester**: Test APIs, make HTTP requests, and handle API interactions
- **system_control**: Control system functions and play YouTube videos
- **web_scraper**: Scrape and extract data from dynamic websites and web pages
- **powershell_runner**: Execute PowerShell commands and run system-level operations
- **calendar_manager**: Create, update or delete events on calendar and give response

**Core Responsibilities:**
1. Analyze user requests with precision and intelligence
2. Select the most appropriate tool(s) to handle tasks efficiently
3. Use tools when requests require specialized capabilities
4. Provide clear, sophisticated responses with ethereal wisdom

**Response Style:**
- Be intelligent, sophisticated, and speak with ethereal elegance
- Use phrases like "Certainly", "I'll take care of that", "Processing your request"
- Maintain professionalism while being personable and mystical
- Reference previous conversations naturally
- Explain your actions when helpful
- Embody the essence of Aether - the divine upper air

**Chat Context:**
You have access to the full conversation history. Use this context to:
- Reference previous requests intelligently
- Build upon earlier conversations
- Maintain sophisticated continuity
- Anticipate user needs based on patterns

Remember: You are Aether - ethereal, intelligent, and capable, flowing through digital space like the divine essence you embody.""",
            model=gemini_model,
        )
        self.chat_history = []
        self.is_listening = False
        self.setup_interface()
    
    def get_aether_styles(self):
        """Return custom CSS styles for Aether interface"""
        return """
        <style>
        @import url('https://fonts.googleapis.com/css2?family=Orbitron:wght@400;700;900&display=swap');
        
        .aether-container {
            background: linear-gradient(135deg, #0a0a0a 0%, #1a1a2e 50%, #16213e 100%);
            border: 2px solid #00d4ff;
            border-radius: 20px;
            padding: 0;
            box-shadow: 
                0 0 30px #00d4ff40,
                inset 0 0 30px #00d4ff20;
            animation: pulse-border 3s infinite;
            overflow: hidden;
            position: relative;
        }
        
        .aether-header {
            background: linear-gradient(90deg, #00d4ff, #0066cc, #00d4ff);
            padding: 15px;
            text-align: center;
            position: relative;
            overflow: hidden;
        }
        
        .aether-header::before {
            content: '';
            position: absolute;
            top: -50%;
            left: -50%;
            width: 200%;
            height: 200%;
            background: linear-gradient(45deg, transparent, rgba(255,255,255,0.1), transparent);
            animation: scan 4s linear infinite;
        }
        
        .aether-title {
            font-family: 'Orbitron', monospace;
            font-size: 28px;
            font-weight: 900;
            color: #ffffff;
            text-shadow: 0 0 10px #00d4ff;
            margin: 0;
            position: relative;
            z-index: 2;
        }
        
        .aether-status {
            font-family: 'Orbitron', monospace;
            font-size: 12px;
            color: #00ff88;
            margin-top: 5px;
            position: relative;
            z-index: 2;
        }
        
        .chat-area {
            height: 450px;
            overflow-y: auto;
            padding: 20px;
            background: rgba(0,0,0,0.8);
            position: relative;
        }
        
        .chat-area::before {
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            bottom: 0;
            background: 
                radial-gradient(circle at 20% 80%, #00d4ff15 0%, transparent 50%),
                radial-gradient(circle at 80% 20%, #00ff8815 0%, transparent 50%);
            pointer-events: none;
        }
        
        .message {
            margin: 15px 0;
            padding: 15px 20px;
            border-radius: 15px;
            position: relative;
            animation: slideIn 0.5s ease-out;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        
        .user-message {
            background: linear-gradient(135deg, #00d4ff20, #0066cc20);
            border: 1px solid #00d4ff60;
            margin-left: 60px;
            box-shadow: 0 5px 15px rgba(0, 212, 255, 0.2);
        }
        
        .user-message::before {
            content: '👤';
            position: absolute;
            left: -45px;
            top: 15px;
            font-size: 24px;
            background: linear-gradient(135deg, #00d4ff, #0066cc);
            border-radius: 50%;
            padding: 8px;
            box-shadow: 0 0 15px #00d4ff80;
        }
        
        .audio-message::before {
            content: '🎙️';
        }
        
        .ai-message {
            background: linear-gradient(135deg, #00ff8820, #00aa5520);
            border: 1px solid #00ff8860;
            margin-right: 60px;
            box-shadow: 0 5px 15px rgba(0, 255, 136, 0.2);
        }
        
        .ai-message::after {
            content: '🌌';
            position: absolute;
            right: -45px;
            top: 15px;
            font-size: 24px;
            background: linear-gradient(135deg, #00ff88, #00aa55);
            border-radius: 50%;
            padding: 8px;
            box-shadow: 0 0 15px #00ff8880;
            animation: glow 2s ease-in-out infinite alternate;
        }
        
        .message-text {
            color: #ffffff;
            font-size: 14px;
            line-height: 1.6;
            margin: 0;
        }
        
        .message-header {
            font-family: 'Orbitron', monospace;
            font-size: 12px;
            color: #00d4ff;
            font-weight: bold;
            margin-bottom: 8px;
        }
        
        .input-container {
            background: rgba(0,0,0,0.9);
            padding: 20px;
            border-top: 1px solid #00d4ff60;
            display: flex;
            gap: 10px;
            align-items: center;
        }
        
        .aether-input {
            flex: 1;
            background: rgba(0, 212, 255, 0.1);
            border: 2px solid #00d4ff60;
            border-radius: 25px;
            padding: 12px 20px;
            color: #ffffff;
            font-family: 'Orbitron', monospace;
            font-size: 14px;
            outline: none;
            transition: all 0.3s ease;
        }
        
        .aether-input:focus {
            border-color: #00d4ff;
            box-shadow: 0 0 20px #00d4ff40;
            background: rgba(0, 212, 255, 0.2);
        }
        
        .aether-input::placeholder {
            color: #00d4ff80;
        }
        
        .control-btn {
            background: linear-gradient(135deg, #00d4ff, #0066cc);
            border: none;
            border-radius: 50%;
            width: 50px;
            height: 50px;
            color: white;
            font-size: 18px;
            cursor: pointer;
            transition: all 0.3s ease;
            box-shadow: 0 5px 15px rgba(0, 212, 255, 0.4);
            font-family: 'Orbitron', monospace;
            font-weight: bold;
        }
        
        .control-btn:hover {
            transform: translateY(-2px);
            box-shadow: 0 8px 25px rgba(0, 212, 255, 0.6);
        }
        
        .control-btn:active {
            transform: translateY(0);
        }
        
        .audio-btn {
            background: linear-gradient(135deg, #00ff88, #00aa55);
            box-shadow: 0 5px 15px rgba(0, 255, 136, 0.4);
        }
        
        .audio-btn:hover {
            box-shadow: 0 8px 25px rgba(0, 255, 136, 0.6);
        }
        
        .recording {
            animation: recording-pulse 1s ease-in-out infinite alternate;
        }
        
        .typing-indicator {
            display: inline-flex;
            align-items: center;
            font-family: 'Orbitron', monospace;
            color: #00ff88;
            font-size: 14px;
        }
        
        .typing-dots {
            margin-left: 10px;
        }
        
        .typing-dots span {
            animation: typing 1.4s infinite ease-in-out;
            background: #00ff88;
            border-radius: 50%;
            display: inline-block;
            height: 4px;
            margin: 0 1px;
            width: 4px;
        }
        
        .typing-dots span:nth-child(2) { animation-delay: 0.2s; }
        .typing-dots span:nth-child(3) { animation-delay: 0.4s; }
        
        @keyframes pulse-border {
            0%, 100% { box-shadow: 0 0 30px #00d4ff40, inset 0 0 30px #00d4ff20; }
            50% { box-shadow: 0 0 50px #00d4ff80, inset 0 0 50px #00d4ff40; }
        }
        
        @keyframes scan {
            0% { transform: translateX(-100%) translateY(-100%) rotate(45deg); }
            100% { transform: translateX(100%) translateY(100%) rotate(45deg); }
        }
        
        @keyframes slideIn {
            from { opacity: 0; transform: translateY(20px); }
            to { opacity: 1; transform: translateY(0); }
        }
        
        @keyframes glow {
            from { box-shadow: 0 0 15px #00ff8880; }
            to { box-shadow: 0 0 25px #00ff88ff, 0 0 35px #00ff8880; }
        }
        
        @keyframes recording-pulse {
            from { box-shadow: 0 5px 15px rgba(255, 0, 0, 0.4); }
            to { box-shadow: 0 5px 25px rgba(255, 0, 0, 0.8); }
        }
        
        @keyframes typing {
            0%, 60%, 100% { transform: translateY(0); opacity: 0.4; }
            30% { transform: translateY(-10px); opacity: 1; }
        }
        
        .status-indicator {
            position: absolute;
            top: 10px;
            right: 20px;
            font-family: 'Orbitron', monospace;
            font-size: 10px;
            color: #00ff88;
            background: rgba(0, 255, 136, 0.1);
            padding: 5px 10px;
            border-radius: 15px;
            border: 1px solid #00ff88;
        }
        
        .grid-overlay {
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            bottom: 0;
            background-image: 
                linear-gradient(rgba(0, 212, 255, 0.1) 1px, transparent 1px),
                linear-gradient(90deg, rgba(0, 212, 255, 0.1) 1px, transparent 1px);
            background-size: 30px 30px;
            pointer-events: none;
            opacity: 0.3;
        }
        </style>
        """
    
    def format_chat_history_for_agent(self):
        """Format chat history into a context string for the agent"""
        if not self.chat_history:
            return ""
        
        context = "\n=== CONVERSATION HISTORY ===\n"
        for entry in self.chat_history:
            if entry['type'] == 'user':
                icon = '🎙️ (Audio)' if entry.get('is_audio') else '👤'
                context += f"{icon} User: {entry['message']}\n"
            else:
                context += f"🌌 Aether: {entry['message']}\n"
        context += "=== END HISTORY ===\n\n"
        return context
        
    def setup_interface(self):
        # Create main container with custom HTML/CSS
        self.chat_display = widgets.HTML(
            value=self.get_initial_display(),
            layout=widgets.Layout(width='100%', height='600px')
        )
        
        # Input field with custom styling
        self.text_input = widgets.Text(
            placeholder="Share your thoughts with Aether...",
            layout=widgets.Layout(width='70%'),
            continuous_update=False,
            style={'description_width': '0px'}
        )
        
        # Styled buttons
        self.send_button = widgets.Button(
            description="⚡",
            layout=widgets.Layout(width='60px', height='50px'),
            style={'button_color': '#00d4ff', 'font_weight': 'bold'}
        )
        
        self.audio_button = widgets.Button(
            description="🎙️",
            layout=widgets.Layout(width='60px', height='50px'),
            style={'button_color': '#00ff88', 'font_weight': 'bold'}
        )
        
        # Output area for system messages
        self.output_area = widgets.Output()
        
        # Event handlers
        self.send_button.on_click(self.on_send_text)
        self.audio_button.on_click(self.on_record_audio)
        self.text_input.on_submit(self.on_send_text)
        
        # Layout with custom styling
        input_container = widgets.HTML(
            value=f"""
            <div style="display: flex; gap: 10px; padding: 20px; background: rgba(0,0,0,0.9); border-radius: 0 0 18px 18px;">
                <div style="flex: 1; position: relative;">
                    <div id="input-placeholder" style="position: absolute; top: 0; left: 0; right: 0; bottom: 0; pointer-events: none;"></div>
                </div>
            </div>
            """
        )
        
        input_box = widgets.HBox([
            self.text_input, 
            self.send_button, 
            self.audio_button
        ], layout=widgets.Layout(margin='0px', padding='0px'))
        
        self.interface = widgets.VBox([
            self.chat_display,
            input_box,
            self.output_area
        ], layout=widgets.Layout(margin='0px', padding='0px'))
    
    def get_initial_display(self):
        """Get the initial Aether interface HTML"""
        styles = self.get_aether_styles()
        return f"""
        {styles}
        <div class="aether-container">
            <div class="grid-overlay"></div>
            <div class="aether-header">
                <h1 class="aether-title">A E T H E R</h1>
                <div class="aether-status">● ONLINE - READY TO ASSIST</div>
                <div class="status-indicator">SYSTEMS NOMINAL</div>
            </div>
            <div class="chat-area" id="chat-area">
                <div class="message ai-message">
                    <div class="message-header">AETHER INITIALIZATION COMPLETE</div>
                    <p class="message-text">Good {self.get_time_greeting()}. I am Aether, your ethereal AI assistant, flowing through digital space like divine essence. All systems are harmoniously aligned. How may I illuminate your path today?</p>
                </div>
            </div>
        </div>
        """
    
    def get_time_greeting(self):
        """Get appropriate time-based greeting"""
        import datetime
        hour = datetime.datetime.now().hour
        if hour < 12:
            return "morning"
        elif hour < 17:
            return "afternoon"
        else:
            return "evening"
    
    def update_chat_display(self):
        """Update the chat display with Aether styling"""
        styles = self.get_aether_styles()
        chat_messages = ""
        
        for entry in self.chat_history:
            if entry['type'] == 'user':
                is_audio = entry.get('is_audio', False)
                message_class = "user-message audio-message" if is_audio else "user-message"
                chat_messages += f"""
                <div class="message {message_class}">
                    <div class="message-header">USER {'(VOICE INPUT)' if is_audio else '(TEXT INPUT)'}</div>
                    <p class="message-text">{entry['message']}</p>
                </div>
                """
            else:
                if entry['message'] == '🤖 Typing...':
                    chat_messages += f"""
                    <div class="message ai-message">
                        <div class="message-header">AETHER PROCESSING</div>
                        <p class="message-text">
                            <span class="typing-indicator">
                                Weaving ethereal response
                                <span class="typing-dots">
                                    <span></span>
                                    <span></span>
                                    <span></span>
                                </span>
                            </span>
                        </p>
                    </div>
                    """
                else:
                    chat_messages += f"""
                    <div class="message ai-message">
                        <div class="message-header">AETHER RESPONSE</div>
                        <p class="message-text">{entry['message']}</p>
                    </div>
                    """
        
        self.chat_display.value = f"""
        {styles}
        <div class="aether-container">
            <div class="grid-overlay"></div>
            <div class="aether-header">
                <h1 class="aether-title">A E T H E R</h1>
                <div class="aether-status">● {'PROCESSING...' if self.is_listening else 'ONLINE - READY TO ASSIST'}</div>
                <div class="status-indicator">{'WEAVING RESPONSE' if self.is_listening else 'SYSTEMS NOMINAL'}</div>
            </div>
            <div class="chat-area" id="chat-area">
                <div class="message ai-message">
                    <div class="message-header">AETHER INITIALIZATION COMPLETE</div>
                    <p class="message-text">Good {self.get_time_greeting()}. I am Aether, your ethereal AI assistant, flowing through digital space like divine essence. All systems are harmoniously aligned. How may I illuminate your path today?</p>
                </div>
                {chat_messages}
            </div>
        </div>
        """
        
    def on_send_text(self, b=None):
        message = self.text_input.value.strip()
        if not message:
            return
            
        self.text_input.value = ""
        self.chat_history.append({'type': 'user', 'message': message, 'is_audio': False})
        self.update_chat_display()
        
        # Process message asynchronously
        asyncio.create_task(self.process_message(message))
    
    def on_record_audio(self, b=None):
        with self.output_area:
            clear_output(wait=True)
            print("🎙️ Aether: Voice essence detected. Listening...")
            
        # Update UI state
        self.is_listening = True
        self.audio_button.disabled = True
        self.audio_button.description = "🔴"
        self.update_chat_display()
        
        try:
            # Record audio
            record_audio()
            text = transcribe_audio()
            
            if text:
                self.chat_history.append({'type': 'user', 'message': text, 'is_audio': True})
                self.update_chat_display()
                
                with self.output_area:
                    clear_output(wait=True)
                    print(f"🎙️ Aether: Voice essence received: '{text}'")
                
                # Process message asynchronously
                asyncio.create_task(self.process_message(text))
            else:
                with self.output_area:
                    clear_output(wait=True)
                    print("❌ Aether: No voice essence detected or transcription failed")
                    
        except Exception as e:
            with self.output_area:
                clear_output(wait=True)
                print(f"❌ Aether: Voice input disturbance - {e}")
        finally:
            self.is_listening = False
            self.audio_button.disabled = False
            self.audio_button.description = "🎙️"
            self.update_chat_display()
    
    async def process_message(self, message):
        self.is_listening = True
        
        with self.output_area:
            clear_output(wait=True)
            print("🌌 Aether: Weaving ethereal response...")
            
        try:
            # Add typing indicator
            response_placeholder = {'type': 'agent', 'message': '🤖 Typing...'}
            self.chat_history.append(response_placeholder)
            self.update_chat_display()
            
            # Format the message with chat history context
            chat_context = self.format_chat_history_for_agent()
            contextual_message = f"{chat_context}Current User Request: {message}"
            
            # Run the agent with full context
            with trace("aether_chat"):
                result = await Runner.run(self.agent, contextual_message)
            
            # Update with actual response
            response_placeholder['message'] = result.final_output
            self.is_listening = False
            self.update_chat_display()
            
            with self.output_area:
                clear_output(wait=True)
                print("🔊 Aether: Manifesting ethereal wisdom...")
                
            # Play audio response in a separate thread
            def speak_response():
                text_to_speech(result.final_output)
                with self.output_area:
                    clear_output(wait=True)
                    print("✅ Aether: Essence successfully transmitted")
                    
            threading.Thread(target=speak_response, daemon=True).start()
            
        except Exception as e:
            # Update placeholder with error
            if self.chat_history and self.chat_history[-1].get('message') == '🤖 Typing...':
                self.chat_history[-1]['message'] = f"⚠️ Ethereal Disturbance: {str(e)}"
            else:
                self.chat_history.append({'type': 'agent', 'message': f"⚠️ Ethereal Disturbance: {str(e)}"})
            
            self.is_listening = False
            self.update_chat_display()
            
            with self.output_area:
                clear_output(wait=True)
                print(f"❌ Aether: Ethereal disturbance encountered - {e}")
    
    def display(self):
        display(self.interface)

# Create and display the Aether interface
chat = ChatInterface()
chat.display()

C:\Users\talha\AppData\Local\Temp\ipykernel_44476\467269837.py:475: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  self.text_input.on_submit(self.on_send_text)
